## 4.5 チューニング

### 共通事前処理

In [1]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# データフレームでの表示精度
pd.options.display.float_format = '{:.4f}'.format
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

### 4.5.1 サンプルデータの読み込み

In [3]:
# サンプルデータの読み込み
# (乳がん疾患データ)

# データのロード
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

# 入力データ: x (30次元)
# 正解データ: y
x = cancer.data
y = cancer.target

In [4]:
# サンプルデータの分割

# データ分割のパラメータ
test_size = 0.1

# データ分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, 
    test_size=test_size, random_state=random_seed,
    stratify=y)

# 分割後サイズ確認
print(x.shape)
print(x_train.shape)
print(x_test.shape)

(569, 30)
(512, 30)
(57, 30)


### 4.5.2 チューニング対象

#### アルゴリズム

In [5]:
# 複数アルゴリズムで精度を比較
# 結果が同じになるようrandom_stateは同一にする

# 線形回帰
from sklearn.linear_model import LogisticRegression
algorism1 = LogisticRegression(random_state=random_seed)

# サポートベクターマシン(カーネル)
from sklearn.svm import SVC
algorism2 = SVC(kernel='rbf', random_state=random_seed)

# 決定木
from sklearn.tree import DecisionTreeClassifier
algorism3 = DecisionTreeClassifier(random_state=random_seed)

# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
algorism4 = RandomForestClassifier(random_state=random_seed)

# XGBoost
from xgboost import XGBClassifier
algorism5 = XGBClassifier(random_state=random_seed)

# アルゴリズムのリスト作成
algorisms = [algorism1, algorism2, algorism3, algorism4, algorism5]

In [6]:
# 複数アルゴリズムで精度比較
for algorism in algorisms:
    
    # 訓練データで学習
    algorism.fit(x_train, y_train)
    
    # 検証データで精度測定
    score = algorism.score(x_test, y_test)
    
    # アルゴリズム名取得
    name = algorism.__class__.__name__

    # 精度とアルゴリズム名表示
    print(f'score: {score:.4f}  {name}')

score: 0.9474  LogisticRegression
score: 0.8947  SVC
score: 0.9474  DecisionTreeClassifier
score: 0.9298  RandomForestClassifier
score: 0.9825  XGBClassifier


#### パラメータチューニング

In [12]:
# デフォルトパラメータの確認
algorism = SVC(kernel='rbf', random_state=random_seed)
print(algorism)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=123, shrinking=True, tol=0.001,
    verbose=False)


In [13]:
# gammaの最適化
algorism = SVC(kernel='rbf', random_state=random_seed)
gammas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for gamma in gammas:
    algorism.gamma = gamma
    algorism.fit(x_train, y_train)
    score = algorism.score(x_test, y_test)
    print(f'score: {score:.4f}  gamma: {gamma}')

score: 0.6316  gamma: 1
score: 0.6316  gamma: 0.1
score: 0.6316  gamma: 0.01
score: 0.9474  gamma: 0.001
score: 0.9474  gamma: 0.0001
score: 0.9474  gamma: 1e-05


In [20]:
# Cの最適化
# gammaは先ほど調べた最適値 0.001を採用
algorism = SVC(kernel='rbf', random_state=random_seed,
    gamma=0.001)

Cs = [1,  10,  100, 1000, 10000]
for C in Cs:
    algorism.C = C
    algorism.fit(x_train, y_train)
    score = algorism.score(x_test, y_test)
    print(f'score: {score:.4f}  C: {C}')

score: 0.9474  C: 1
score: 0.9298  C: 10
score: 0.9298  C: 100
score: 0.9298  C: 1000
score: 0.9298  C: 10000


### 4.5.3 チューニング方法

#### 交差検定

In [21]:
# 特定のアルゴリズムに対して交差検定を実施

# アルゴリズムの定義
algorism = SVC(kernel='rbf',random_state=random_seed,
    gamma=0.001, C=1)

# 分割時に正解データの分布が偏らないようにStratifiedKFoldを利用
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

# 交差検定の実施 (分割数=3)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(algorism , x_train, y_train, cv=stratifiedkfold)

# 平均値の計算
mean = scores.mean()

# 結果表示
print(f'平均スコア: {mean:.4f}  個別スコア: {scores}')

平均スコア: 0.9141  個別スコア: [0.8889 0.9181 0.9353]


In [22]:
# 候補アルゴリズムのリスト作成

from sklearn.linear_model import LogisticRegression
algorism1 = LogisticRegression(random_state=random_seed)

from sklearn.svm import SVC
algorism2 = SVC(kernel='rbf',random_state=random_seed,
    gamma=0.001, C=1)

from sklearn.tree import DecisionTreeClassifier
algorism3 = DecisionTreeClassifier(random_state=random_seed)

from sklearn.ensemble import RandomForestClassifier
algorism4 = RandomForestClassifier(random_state=random_seed)

from xgboost import XGBClassifier
algorism5 = XGBClassifier(random_state=random_seed)

algorisms = [algorism1, algorism2, algorism3, algorism4, algorism5]

In [24]:
# 複数アルゴリズムで精度を比較

# 分割時に正解データの分布が偏らないようにStratifiedKFoldを利用
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score
for algorism in algorisms:
    # 交差検定法の実行
    scores = cross_val_score(algorism , x_train, y_train, cv=stratifiedkfold)
    score = scores.mean()
    name = algorism.__class__.__name__
    print(f'平均スコア: {score:.4f}  個別スコア: {scores}  {name}')

平均スコア: 0.9473  個別スコア: [0.9415 0.9474 0.9529]  LogisticRegression
平均スコア: 0.9141  個別スコア: [0.8889 0.9181 0.9353]  SVC
平均スコア: 0.9062  個別スコア: [0.8713 0.9415 0.9059]  DecisionTreeClassifier
平均スコア: 0.9629  個別スコア: [0.9649 0.9591 0.9647]  RandomForestClassifier
平均スコア: 0.9590  個別スコア: [0.9591 0.9649 0.9529]  XGBClassifier


#### グリッドサーチ

In [18]:
# グリッドサーチを交差検定を組み合わせて最適なパラメータを探索
params = {
      'C':[1, 10, 100, 1000, 10000],
      'gamma':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
}
algorism = SVC(random_state=random_seed)

from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(algorism, params, cv=stratifiedkfold)
gs.fit(x_train, y_train)

# ベストのモデルを取得し検証データを分類
best = gs.best_estimator_
best_pred = best.predict(x_test)
print(best)

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1e-05, kernel='rbf',
    max_iter=-1, probability=False, random_state=123, shrinking=True, tol=0.001,
    verbose=False)


In [19]:
# 精度の取得
score = best.score(x_test, y_test)
print(f'スコア: {score:.4f}')

# 混同行列を出力
from sklearn.metrics import confusion_matrix
print()
print('混同行列')
print(confusion_matrix(y_test, best_pred))

スコア: 0.9825

混同行列
[[20  1]
 [ 0 36]]
